In [19]:
from pyspark.sql import SparkSession


spark = SparkSession.builder \
.appName('Spark SQL Managed vs External') \
.enableHiveSupport() \
.getOrCreate()

25/02/05 11:30:05 INFO SparkEnv: Registering MapOutputTracker
25/02/05 11:30:05 INFO SparkEnv: Registering BlockManagerMaster
25/02/05 11:30:05 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
25/02/05 11:30:06 INFO SparkEnv: Registering OutputCommitCoordinator


In [20]:
df = spark.read\
.format('csv')\
.option('inferScema',"True")\
.option('header','True')\
.load('/data/customers_100.csv')

In [21]:
df.show(5)

+-----------+----------+---------+-----------+-------+-----------------+---------+
|customer_id|      name|     city|      state|country|registration_date|is_active|
+-----------+----------+---------+-----------+-------+-----------------+---------+
|          0|Customer_0|     Pune|Maharashtra|  India|       2023-06-29|    False|
|          1|Customer_1|Bangalore| Tamil Nadu|  India|       2023-12-07|     True|
|          2|Customer_2|Hyderabad|    Gujarat|  India|       2023-10-27|     True|
|          3|Customer_3|Bangalore|  Karnataka|  India|       2023-10-17|    False|
|          4|Customer_4|Ahmedabad|  Karnataka|  India|       2023-03-14|    False|
+-----------+----------+---------+-----------+-------+-----------------+---------+
only showing top 5 rows



In [23]:
df.createOrReplaceTempView('temp_customers')

In [24]:
spark.sql('select * from temp_customers limit 5').show()

+-----------+----------+---------+-----------+-------+-----------------+---------+
|customer_id|      name|     city|      state|country|registration_date|is_active|
+-----------+----------+---------+-----------+-------+-----------------+---------+
|          0|Customer_0|     Pune|Maharashtra|  India|       2023-06-29|    False|
|          1|Customer_1|Bangalore| Tamil Nadu|  India|       2023-12-07|     True|
|          2|Customer_2|Hyderabad|    Gujarat|  India|       2023-10-27|     True|
|          3|Customer_3|Bangalore|  Karnataka|  India|       2023-10-17|    False|
|          4|Customer_4|Ahmedabad|  Karnataka|  India|       2023-03-14|    False|
+-----------+----------+---------+-----------+-------+-----------------+---------+



In [27]:
spark.sql('show tables').show()

+---------+--------------+-----------+
|namespace|     tableName|isTemporary|
+---------+--------------+-----------+
|         |temp_customers|       true|
+---------+--------------+-----------+



In [26]:
spark.sql('drop table customers').show()

++
||
++
++



In [30]:
spark.sql('create table managed_customers as select * from temp_customers')

25/02/05 11:34:42 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
25/02/05 11:34:42 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


DataFrame[]

In [31]:
spark.sql('describe extended managed_customers').show(truncate=False)

+----------------------------+---------------------------------------------------------+-------+
|col_name                    |data_type                                                |comment|
+----------------------------+---------------------------------------------------------+-------+
|customer_id                 |string                                                   |null   |
|name                        |string                                                   |null   |
|city                        |string                                                   |null   |
|state                       |string                                                   |null   |
|country                     |string                                                   |null   |
|registration_date           |string                                                   |null   |
|is_active                   |string                                                   |null   |
|                            |

In [32]:
!hadoop fs -ls /user/hive/warehouse/

Found 2 items
drwxr-xr-x   - root hadoop          0 2025-02-05 10:40 /user/hive/warehouse/ecommerce.db
drwxr-xr-x   - root hadoop          0 2025-02-05 11:34 /user/hive/warehouse/managed_customers


# External Table

In [45]:
!hadoop fs -ls /data/

Found 13 items
-rw-r--r--   2 root hadoop  343317147 2025-02-03 11:53 /data/customers.csv
-rw-r--r--   2 root hadoop       5488 2025-02-03 11:53 /data/customers_100.csv
-rw-r--r--   2 root hadoop   10528211 2025-02-03 11:53 /data/customers_10mb.csv
-rw-r--r--   2 root hadoop    1060750 2025-02-03 11:53 /data/customers_1mb.csv
-rw-r--r--   2 root hadoop  570783961 2025-02-03 11:53 /data/customers_500mb.csv
-rw-r--r--   2 root hadoop        351 2025-02-04 05:35 /data/customers_with_errors.csv
-rw-r--r--   2 root hadoop         23 2025-02-04 14:53 /data/date.csv
-rw-r--r--   2 root hadoop        209 2025-02-05 06:28 /data/dates_data.csv
drwxr-xr-x   - root hadoop          0 2025-02-05 11:43 /data/external_data
drwxr-xr-x   - root hadoop          0 2025-02-04 07:53 /data/write_output.csv
drwxr-xr-x   - root hadoop          0 2025-02-04 07:56 /data/write_output_1_part
drwxr-xr-x   - root hadoop          0 2025-02-04 07:59 /data/write_output_1_part_2
drwxr-xr-x   - root hadoop          0 202

In [38]:
!hadoop fs -head /data/external_data/customers_1mb.csv

customer_id,name,city,state,country,registration_date,is_active
0,Customer_0,Pune,Maharashtra,India,2023-06-29,False
1,Customer_1,Bangalore,Tamil Nadu,India,2023-12-07,True
2,Customer_2,Hyderabad,Gujarat,India,2023-10-27,True
3,Customer_3,Bangalore,Karnataka,India,2023-10-17,False
4,Customer_4,Ahmedabad,Karnataka,India,2023-03-14,False
5,Customer_5,Hyderabad,Karnataka,India,2023-07-28,False
6,Customer_6,Pune,Delhi,India,2023-08-29,False
7,Customer_7,Ahmedabad,West Bengal,India,2023-12-28,True
8,Customer_8,Pune,Karnataka,India,2023-06-22,True
9,Customer_9,Mumbai,Telangana,India,2023-01-05,True
10,Customer_10,Pune,Gujarat,India,2023-08-05,True
11,Customer_11,Delhi,West Bengal,India,2023-08-02,False
12,Customer_12,Chennai,Gujarat,India,2023-11-21,False
13,Customer_13,Chennai,Karnataka,India,2023-11-06,True
14,Customer_14,Hyderabad,Tamil Nadu,India,2023-02-07,False
15,Customer_15,Mumbai,Gujarat,India,2023-03-02,True
16,Customer_16,Chennai,Karnataka,India,2023-04-05,False
17,Customer_17,Hyd

In [57]:
spark.sql('''
create external table external_customers_2 (
customer_id INT,
name STRING,
city STRING,
state string,
country string,
registration_date STRING,
is_active BOOLEAN
)
using csv
location '/data/external_data/'
''')


25/02/05 11:55:14 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider csv. Persisting data source table `default`.`external_customers_2` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


DataFrame[]

In [41]:
spark.sql('describe extended external_customers_2').show(truncate=False)

+----------------------------+--------------------------------------------------+-------+
|col_name                    |data_type                                         |comment|
+----------------------------+--------------------------------------------------+-------+
|customer_id                 |int                                               |null   |
|name                        |string                                            |null   |
|city                        |string                                            |null   |
|state                       |string                                            |null   |
|country                     |string                                            |null   |
|registration_date           |string                                            |null   |
|is_active                   |boolean                                           |null   |
|                            |                                                  |       |
|# Detaile

In [44]:
spark.sql('select * from external_customers_2 limit 5').show()

+-----------+----------+---------+-----------+-------+-----------------+---------+
|customer_id|      name|     city|      state|country|registration_date|is_active|
+-----------+----------+---------+-----------+-------+-----------------+---------+
|       null|      name|     city|      state|country|registration_date|     null|
|          0|Customer_0|     Pune|Maharashtra|  India|       2023-06-29|    false|
|          1|Customer_1|Bangalore| Tamil Nadu|  India|       2023-12-07|     true|
|          2|Customer_2|Hyderabad|    Gujarat|  India|       2023-10-27|     true|
|          3|Customer_3|Bangalore|  Karnataka|  India|       2023-10-17|    false|
+-----------+----------+---------+-----------+-------+-----------------+---------+



In [46]:
!hadoop fs -ls /user/hive/warehouse/

Found 2 items
drwxr-xr-x   - root hadoop          0 2025-02-05 10:40 /user/hive/warehouse/ecommerce.db
drwxr-xr-x   - root hadoop          0 2025-02-05 11:34 /user/hive/warehouse/managed_customers


In [47]:
spark.sql('show tables').show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|  default|  external_customers|      false|
|  default|external_customers_2|      false|
|  default|   managed_customers|      false|
|         |      temp_customers|       true|
+---------+--------------------+-----------+



In [48]:
spark.sql('drop table managed_customers').show()

++
||
++
++



In [49]:
!hadoop fs -ls /user/hive/warehouse/

Found 1 items
drwxr-xr-x   - root hadoop          0 2025-02-05 10:40 /user/hive/warehouse/ecommerce.db


In [50]:
spark.sql('select * from managed_customers').show()

AnalysisException: Table or view not found: managed_customers; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [managed_customers], [], false


In [51]:
!hadoop fs -ls /data/external_data    

Found 1 items
-rw-r--r--   2 root hadoop    1060750 2025-02-05 11:43 /data/external_data/customers_1mb.csv


In [54]:
spark.sql('drop table external_customers_2').show()

++
||
++
++



In [53]:
!hadoop fs -ls /data/external_data    

Found 1 items
-rw-r--r--   2 root hadoop    1060750 2025-02-05 11:43 /data/external_data/customers_1mb.csv


In [55]:
!hadoop fs -ls /data

Found 13 items
-rw-r--r--   2 root hadoop  343317147 2025-02-03 11:53 /data/customers.csv
-rw-r--r--   2 root hadoop       5488 2025-02-03 11:53 /data/customers_100.csv
-rw-r--r--   2 root hadoop   10528211 2025-02-03 11:53 /data/customers_10mb.csv
-rw-r--r--   2 root hadoop    1060750 2025-02-03 11:53 /data/customers_1mb.csv
-rw-r--r--   2 root hadoop  570783961 2025-02-03 11:53 /data/customers_500mb.csv
-rw-r--r--   2 root hadoop        351 2025-02-04 05:35 /data/customers_with_errors.csv
-rw-r--r--   2 root hadoop         23 2025-02-04 14:53 /data/date.csv
-rw-r--r--   2 root hadoop        209 2025-02-05 06:28 /data/dates_data.csv
drwxr-xr-x   - root hadoop          0 2025-02-05 11:43 /data/external_data
drwxr-xr-x   - root hadoop          0 2025-02-04 07:53 /data/write_output.csv
drwxr-xr-x   - root hadoop          0 2025-02-04 07:56 /data/write_output_1_part
drwxr-xr-x   - root hadoop          0 2025-02-04 07:59 /data/write_output_1_part_2
drwxr-xr-x   - root hadoop          0 202

In [56]:
spark.sql('show tables').show()

+---------+--------------+-----------+
|namespace|     tableName|isTemporary|
+---------+--------------+-----------+
|         |temp_customers|       true|
+---------+--------------+-----------+



In [58]:
spark.stop()